In [24]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [25]:
first = pd.read_csv('document.txt', sep=r'\s+')
first = first.drop('ind', axis=1)
# first = first[first['product']>0]
second = pd.read_csv('1.txt', sep=r'\s+')
second = second.drop('ind', axis=1)
third = pd.read_csv('2.txt', sep=r'\s+')
third = third.drop('ind', axis=1)

In [26]:
#Небольшая процедура для предварительного анализа данных
def define_dataset(df):
      print(df.shape)
      print(df.info())
      print(df.head(40))
      print(df.describe())
      return df['id'].value_counts().head(20)

features_1 = first.drop(['id','product'], axis=1)
features_ohe_1 = pd.get_dummies(features_1, drop_first=True)
target_1 = first['product']

features_2 = second.drop(['id','product'], axis=1)
features_ohe_2 = pd.get_dummies(features_2, drop_first=True)
target_2 = second['product']

features_3 = third.drop(['id','product'], axis=1)
features_ohe_3 = pd.get_dummies(features_3, drop_first=True)
target_3 = third['product']

#Разбиваем данные на обучающую и валидационную выборки в соотношении 75:25.
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_ohe_1, target_1, test_size=0.25, random_state=12345)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_ohe_2, target_2, test_size=0.25, random_state=12345)
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_ohe_3, target_3, test_size=0.25, random_state=12345)
model = LinearRegression() #Применяем модель линейной регрессии

#Признаки кодируем во избежание доминирования одного из них
numeric = ['f0','f1','f2']
scaler = StandardScaler()
def scale(features_train, features_valid, numeric):
      scaler.fit(features_train_1[numeric])
      features_train[numeric] = scaler.transform(features_train[numeric])
      features_valid[numeric] = scaler.transform(features_valid[numeric])
      return

#Обучаем модель и проводим предсказания на первой валидационной выборке.
def study(features_train, features_valid, target_train, target_valid, number_location):
      model.fit(features_train,target_train) # обучите модель на первой тренировочной выборке
      predictions_valid = model.predict(features_valid) # получите  предсказания модели на первой валидационной выборке
      #Выводим на печать средний запас предсказанного сырья и RMSE модели для первой локации.
      mse = mean_squared_error(target_valid, predictions_valid)

      # < извлекаем корень из MSE >
      result =  mse ** 0.5
      print("Средний запас предсказанного на валидационной выборке", number_location, "сырья:", predictions_valid.mean(), '(тыс. баррелей)')
      print("RMSE модели линейной регрессии на валидационной выборке", number_location, ":", result)
      return predictions_valid

In [27]:
scale(features_train_1, features_valid_1, numeric)
scale(features_train_2, features_valid_2, numeric)
scale(features_train_3, features_valid_3, numeric)

In [28]:
study(features_train_1, features_valid_1, target_train_1, target_valid_1, 1)

Средний запас предсказанного на валидационной выборке 1 сырья: 101.57898111447824 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке 1 : 1.1414291393555036


array([135.66029436,  83.40132081, 135.85984776, 136.15562042,
        83.1414788 ,  55.74472549, 138.9568881 , 107.85486195,
         3.90662616, 135.10814729])

In [29]:
study(features_train_2, features_valid_2, target_train_2, target_valid_2, 2)

Средний запас предсказанного на валидационной выборке 2 сырья: 79.60799856167405 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке 2 : 39.35239879630726


array([86.25531811, 94.42212813, 54.78362276, 78.82098344, 83.11798094,
       69.22219427, 85.62882546, 76.97265683, 69.07739358, 97.7788821 ])

In [30]:
study(features_train_3, features_valid_3, target_train_3, target_valid_3, 3)

Средний запас предсказанного на валидационной выборке 3 сырья: 99.64955863171336 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке 3 : 45.50537343061165


array([121.36420993,  88.19996895,  90.82085875, 109.21275929,
        83.80523339, 105.59403231, 108.2624712 , 103.85781818,
        93.75486443,  91.62336988])